In [1]:
import PARAMS
import numpy as np
import torch
import experiment

In [2]:
exp = experiment.Experiment(PARAMS.d, PARAMS.k, PARAMS.nT, PARAMS.n1, PARAMS.n2)
exp.instantiate()

In [5]:
class TaskPredict(torch.nn.Module):
    def __init__(self):
        super(TaskPredict, self).__init__()
        self.w = torch.nn.Parameter(torch.randn(PARAMS.k))

    def forward(self, x):
        return x @ self.w

In [6]:
class MTLDLR(torch.nn.Module):
    def __init__(self, tasks):
        super(MTLDLR, self).__init__()
        self.B = torch.nn.Parameter(torch.randn(PARAMS.d, PARAMS.k))
        self.W = torch.nn.ModuleDict({
            task_id: TaskPredict() for task_id in tasks
        })

    def forward(self, x: dict):
        return {
            task_id: self.W[task_id](x[task_id] @ self.B) for task_id in self.W
            }

In [48]:
model = MTLDLR([t.task_id for t in exp.source_tasks])
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [11]:
source_X = {t.task_id: t.X for t in exp.source_tasks}
source_y = {t.task_id: t.y for t in exp.source_tasks}

In [49]:
prediction = model(source_X)
def Loss(prediction, true_y):
    l = torch.cat(tuple(prediction.values())) - torch.cat(tuple(true_y.values()))
    return torch.mean(l**2) / 2

In [50]:
for epoch in range(1000):
    model.zero_grad()
    prediction = model(source_X)
    loss = Loss(prediction, source_y)
    loss.backward()
    optimizer.step()
    if (epoch == 0) or (epoch % 100 == 0):
        print(loss)

tensor(985.7751, grad_fn=<DivBackward0>)
tensor(61.1624, grad_fn=<DivBackward0>)
tensor(10.7805, grad_fn=<DivBackward0>)
tensor(4.7310, grad_fn=<DivBackward0>)
tensor(3.7720, grad_fn=<DivBackward0>)
tensor(3.5168, grad_fn=<DivBackward0>)
tensor(3.3433, grad_fn=<DivBackward0>)
tensor(3.1682, grad_fn=<DivBackward0>)
tensor(3.0068, grad_fn=<DivBackward0>)
tensor(2.8641, grad_fn=<DivBackward0>)
